##### Initialize

In [1]:
from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import logging

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

## Secret Keys for Bolddesk
bd_api_key = kv_client.get_secret('bolddesk-nera-it-api-key').value
bd_base_url = kv_client.get_secret('bolddesk-nera-it-api-base-url').value

In [2]:
from module.azure_ad import AzureAD
from module.bolddesk import Bolddesk

## Initialize AD and Bolddesk API Module
ad = AzureAD(tenant_id, client_id, client_secret)
bd = Bolddesk(bd_base_url, bd_api_key)

##### add_new_user_to_bd

In [ ]:
logging.info('started: add_new_user_to_bd()')
## Add New AD User to Bolddesk Contact
## New AD User Defiend as
## - userPurpose is 'user'
## - Member
## - accountEnabled
## - Not exist in Bolddesk

## AD users to be added to Bolddesk
all_employees_df = ad.list_users().query('userPurpose=="user" and userType=="Member" and accountEnabled=="TRUE"').set_index('userPrincipalName')
employees_to_bd_index = ~all_employees_df.index.isin(bd.list_users().index) ## not in bolddesk
employees_to_bd_df    = all_employees_df.loc[employees_to_bd_index]

## Add new contact in Bolddesk as verified
for emailId, new_user in employees_to_bd_df.iterrows():
    ## map AD fields to Bolddesk fields
    new_contact = {
        'emailId'              : emailId,
        'contactName'          : new_user.get('displayName'),
        'contactDisplayName'   : new_user.get('displayName'),
        'contactMobileNo'      : new_user.get('mobilePhone'),
        'contactJobTitle'      : new_user.get('jobTitle'),
        'cf_contactCountry'       : new_user.get('country'),
        'cf_contactCity'          : new_user.get('city'),
        'cf_contactManagerEmailId': new_user.get('manager_userPrincipalName'),
        'isVerified'              : True  ## force verified, so that users don't get invitation email from Bolddesk
    }
    logging.info(f'Adding new contact: {emailId}, {new_contact}')
    bd.add_contact(new_contact)

##### update_bd_contact_on_ad_changes

In [ ]:
logging.info('started: update_bd_contact_on_ad_changes()')
## Update Bolddesk Existing Contacts with Data From Azure AD Users

## Define AD-Bolddesk fields map
## note: contactMobileNo and contactJobTitle is not available in contacts_df, hence can't map here
ad_bd_fields_map = {
    'userPrincipalName': 'emailId',
    'displayName'      : 'contactName',
    'country'          : 'cf_contactCountry',
    'mobilePhone'      : 'contactMobileNo',
    'jobTitle'         : 'contactJobTitle',
    'city'             : 'cf_contactCity',
    'manager_userPrincipalName': 'cf_contactManagerEmailId'
}

## Normalize AD Users
ad_users_df = ad.list_users().rename( columns=ad_bd_fields_map)
ad_users_df = ad_users_df.loc[:, ad_bd_fields_map.values()].set_index('emailId').fillna('')

## Normalize Contacts
contacts_df = bd.list_contacts().set_index('emailId')

## Select Existing Contacts that Exist in Azure AD
common_contacts = contacts_df.index.isin(ad_users_df.index)

## Loop Through Intersection To Check Sync
for emailId, contact in contacts_df.loc[common_contacts].iterrows():
    
    user_dict    = ad_users_df.loc[emailId].to_dict()    
    contact_dict = bd.filter_dict_by_keys(contact.to_dict(), user_dict.keys())
    diff = bd.dict_diff(contact_dict, user_dict)
    
    ## difference detected
    if diff:
        
        ## Construct the Update Record
        change_dict = {}
        for key, value in diff.items():
            change_dict[key] = value[1]

        ## Update the Contact
        bd.update_contact(contact.userId, change_dict)            
        # logging.info(f'Updating contact: {emailId}/{contact.userId}: {change_dict}, contact_dict: {contact_dict}, user_dict: {user_dict}')
        logging.info(f'Updating contact: {emailId}/{contact.userId}: {change_dict}')

##### block_invalid_bd_contacts

In [ ]:
logging.info('started: block_invalid_bd_contacts()')
## Block Bolddesk Users

## Find Out All Non Blcoked Contacts That Are Not Exist in Azure AD
## Valid Contacts are defined as (all condition applied)
## - exist in Azure AD 
## - Azure AD accountEnabled
## - Azure AD Member
## - Active Contact - not blocked or Deleted

## Exception, mark as non spammer
valid_non_nera_domains = ['central.sophos.com']

ad_users_df = ad.list_users().query('userPurpose=="user" and userType=="Member" and accountEnabled=="TRUE"').set_index('userPrincipalName')
contacts_df = bd.list_contacts().set_index('emailId')
# invalid_contacts = ~contacts_df.index.isin(ad_users_df.index) & ~ (contacts_df.isBlocked | contacts_df.isDeleted) & ~contacts_df.index.str.contains('central.sophos.com')
invalid_contacts = ~contacts_df.index.isin(ad_users_df.index) & ~ (contacts_df.isBlocked | contacts_df.isDeleted) & ~contacts_df.index.isin(valid_non_nera_domains)
invalid_contacts_df = contacts_df.loc[invalid_contacts]

## Invalid Nera Staff, block if the number is <5% of total AD users
is_nera = invalid_contacts_df.index.str.contains('@nera.net')
nera_invalid_df = invalid_contacts_df[is_nera]
if sum(is_nera)/len(ad.users_df) < 0.05:
    for emailId, contact in nera_invalid_df.iterrows():
        print('Blocking Bolddesk Nera Contact Not in AD: ', emailId)
        bd.block_contact(contact.userId, markTicketAsSpam=False)

## Invalid Spammer (non Nera email)
is_not_nera = ~is_nera
spammer_invalid_df = invalid_contacts_df[is_not_nera]
if sum(is_not_nera)/len(ad.users_df) < 0.05:
    for emailId, contact in spammer_invalid_df.iterrows():
        print('Blocking Bloddesk Spammer Contact: ', emailId)
        bd.block_contact(contact.userId, markTicketAsSpam=True)    

##### save_to_warehouse

In [7]:
from sqlalchemy import create_engine, MetaData, Table
import urllib, struct
from math import floor 

logging.info('started: save_to_warehouse()')
token = def_credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(token)}s', len(token), token)

## Establish Azure SQL Connection using pyodbc
server="nera-sql.database.windows.net"
database="nera_db"
driver="{ODBC Driver 17 for SQL Server}"
connection_string = 'DRIVER='+driver+';SERVER='+server+';DATABASE='+database
params = urllib.parse.quote(connection_string)
SQL_COPT_SS_ACCESS_TOKEN = 1256
db_engine = create_engine("mssql+pyodbc:///?odbc_connect={0}".format(params), connect_args={'attrs_before': {SQL_COPT_SS_ACCESS_TOKEN:token_struct}})
conn = db_engine.connect()
metadata = MetaData()

## Define the save jobs
save_list = [
    ## log_message, table_name, function_name
    # ('save_to_warehouse(): saving ad_users',                  'ad_users',           'list_users'),
    # ('save_to_warehouse(): saving saving ad_groups',          'ad_groups',          'list_groups'),
    # ('save_to_warehouse(): saving saving ad_groups_members',  'ad_groups_members',  'list_groups_members'),
    # ('save_to_warehouse(): saving saving ad_groups_owners',   'ad_groups_owners',   'list_groups_owners'),
    # ('save_to_warehouse(): saving saving ad_devices_users',   'ad_devices_users',   'list_devices_users'),
    ('save_to_warehouse(): saving authentications',           'ad_auth_details',    'list_auth_details'),
]

## run all save jobs
for job in save_list:

    logging.info(job[0])

    ## delete all rows
    my_table = Table(job[1], metadata, autoload_with=db_engine)
    delete_stmt = my_table.delete()
    conn.execute(delete_stmt)
    conn.commit()
    
    ## get new data and save to SQL
    list_func = getattr(ad, job[2])
    df = list_func()
    chunksize = floor(2100/df.shape[1]) -1
    df.to_sql(job[1], con=db_engine, index=False, if_exists='append', method='multi', chunksize=chunksize)


##### Batch Job

In [ ]:
## Batch Update Contacts timezoneId based on cf_contactCountry
# bd = Bolddesk(base_url, api_key)
# for emailId, contact in bd.contacts_df.iterrows():
#     country = contact.cf_contactCountry
#     city    = contact.cf_contactCity
#     timezone_id = bd.get_timezone_id(country, city)
#     bd.update_contact(contact.userId, {'timezoneId': timezone_id})

In [ ]:
# ## Update Bolddesk Existing Contacts with Data From Azure AD Users

# ## Define AD-Bolddesk fields map
# ## note: contactMobileNo and contactJobTitle is not available in contacts_df, hence can't map here
# bd_ad_fields_map = {
#     'userPrincipalName': 'emailId',
#     'displayName'      : 'contactName',
#     'country'          : 'cf_contactCountry',
#     'mobilePhone'      : 'contactMobileNo',
#     'jobTitle'         : 'contactJobTitle',
#     'city'             : 'cf_contactCity',
#     'manager_userPrincipalName': 'cf_contactManagerEmailId'
# }

# ## Normalize AD Users
# ad_users_df = ad.list_users().rename( columns=bd_ad_fields_map)
# ad_users_df = ad_users_df.loc[:, bd_ad_fields_map.values()].set_index('emailId')

# ## Normalize Contacts
# contacts_df = bd.list_contacts().set_index('emailId')

# ## Select Existing Contacts that Exist in Azure AD
# common_contacts = contacts_df.index.isin(ad_users_df.index)

# ## Loop Through Intersection To Check Sync
# for emailId, contact in contacts_df.loc[common_contacts].iterrows():
    
#     user_dict    = ad_users_df.loc[emailId].to_dict()    
#     contact_dict = bd.filter_dict_by_keys(contact.to_dict(), user_dict.keys())
#     diff = bd.dict_diff(contact_dict, user_dict)
    
#     ## difference detected
#     if diff:
        
#         ## Construct the Update Record
#         change_dict = {}
#         for key, value in diff.items():
#             change_dict[key] = value[1]

#         ## Update the Contact
#         bd.update_contact(contact.userId, change_dict)            
#         logging.info(f'Updating contact: {emailId}/{contact.userId}: {change_dict}')

##### Test

In [3]:
au = ad.list_auth_details()

In [4]:
au.iloc[0]

id                                                            87c0f890-5601-4171-933b-05fa71192733
userPrincipalName                                                               aaron.lim@nera.net
userDisplayName                                                                          Aaron Lim
userType                                                                                    member
isAdmin                                                                                      False
isSsprRegistered                                                                              True
isSsprEnabled                                                                                 True
isSsprCapable                                                                                 True
isMfaRegistered                                                                               True
isMfaCapable                                                                                  True
isPassword

0                                                    []
1     [{'@odata.type': '#microsoft.graph.servicePrin...
2     [{'@odata.type': '#microsoft.graph.servicePrin...
3     [{'@odata.type': '#microsoft.graph.user', 'id'...
4     [{'@odata.type': '#microsoft.graph.user', 'id'...
5                                                    []
6                                                    []
7     [{'@odata.type': '#microsoft.graph.user', 'id'...
8     [{'@odata.type': '#microsoft.graph.user', 'id'...
9     [{'@odata.type': '#microsoft.graph.user', 'id'...
10    [{'@odata.type': '#microsoft.graph.user', 'id'...
11                                                   []
12                                                   []
13    [{'@odata.type': '#microsoft.graph.user', 'id'...
14                                                   []
15    [{'@odata.type': '#microsoft.graph.servicePrin...
16                                                   []
17                                              